# IMPORTS

In [2]:
import pandas as pd

In [25]:
df = pd.read_csv("nuclear_dataset.csv", usecols=['C1', 'C2', 'C3', 'C4'])

# Informacje preferencyjne

W ramach informacji preferencyjnych należy ocenić parami warianty 0 i 15 (W skutek wyczerpywania się złóż materiałów radioaktywnych, elektrownie mogą zacząć korzystać z bardziej ubogich złóż, co przełoży się na zwiększenie ilości odpadów radioaktywnych) oraz 10 i 17 (Inflacja może znacząco wzrosnąć w przyszłości, więc minimalizacja kosztów w przyszłości powinny być ważniejsza niż koszty aktualne)

In [26]:
df[df.index.isin([0, 15, 10, 17])]

,C1,C2,C3,C4
0,0.60,0.93,0.00,0.73
10,0.61,0.54,0.38,0.49
15,0.44,0.95,0.00,0.90
17,0.76,0.06,1.00,0.60


Wariant 0 > Wariant 15 - $ U(W0) > U(W15) $

Wariant 10 > Wariant 17 - $ U(W10) > U(W17) $



In [41]:
preferences = [
    [0, 15],  # 0 > 15
    [10, 17]  # 10 > 17
]

In [23]:
from pulp import *

In [72]:
# Utworzenie problemu UTA-GMS
problem = LpProblem("uta-gms", LpMaximize)

# Utworzenie zmiennej maksymalizowanej
epsilon = LpVariable("epsilon")

# Utworzenie zmiennych decyzyjnych dla każdego kryterium w problemie programowania matematycznego
decision_variables = {}
for column in df.columns:
    unique_values = df[column].sort_values().unique()
    decision_variables[column] = [LpVariable(f"x_{column}_{value}", 0, 1) for value in unique_values]

# Dodanie ograniczenia na normalizację (maksymalna użyteczność globalna równa 1)
# Ponieważ wszystkie kryteria są typu koszt, możemy założyć, że wariant o minimalnych wartościach jest najlepszy
problem += lpSum([x[0] for x in decision_variables.values()]) == 1

# Dodanie ograniczenia na normalizację (minimalna użyteczność globalna równa 0)
# Ponieważ wszystkie kryteria są typu koszt, możemy założyć, że wariant o maksymalnych wartościach jest najgorszy
problem += lpSum([x[-1] for x in decision_variables.values()]) == 0

# Dodanie ograniczeń na monotoniczność funkcji cząstkowych
# W problemie używana jest funkcja ogólna, więc nie ma potrzeby wyliczania interpolacji liniowej
# Wystarczy dodać ograniczenia, które gwarantują, że kolejne zmienne decyzyjne mają nierosnące wartości użyteczności
for column in df.columns:
    for i in range(len(decision_variables[column])-1):
        problem += decision_variables[column][i] >= decision_variables[column][i+1]

# Dodanie ograniczeń związanych z preferencjami decydenta
for preference in preferences:
    alt_1 = df.loc[preference[0]].to_dict()
    alt_1_variables = [next((variable for variable in decision_variables[k] if str(variable) == f"x_{k}_{v}"), None) for k, v in alt_1.items()]
        
    alt_2 = df.loc[preference[1]].to_dict()
    alt_2_variables = [next((variable for variable in decision_variables[k] if str(variable) == f"x_{k}_{v}"), None) for k, v, in alt_2.items()]

    print(alt_1_variables, alt_2_variables)

    problem += lpSum(alt_1_variables) >= lpSum(alt_2_variables) + epsilon

[x_C1_0.6, x_C2_0.93, x_C3_0.0, x_C4_0.73] [x_C1_0.44, x_C2_0.95, x_C3_0.0, x_C4_0.9]
[x_C1_0.61, x_C2_0.54, x_C3_0.38, x_C4_0.49] [x_C1_0.76, x_C2_0.06, x_C3_1.0, x_C4_0.6]


In [73]:
for constraint in problem.constraints.values():
    print(constraint)

x_C1_0.32 + x_C2_0.03 + x_C3_0.0 + x_C4_0.49 = 1
x_C1_1.0 + x_C2_1.0 + x_C3_1.0 + x_C4_1.0 = 0
x_C1_0.32 - x_C1_0.34 >= 0
x_C1_0.34 - x_C1_0.35 >= 0
x_C1_0.35 - x_C1_0.4 >= 0
x_C1_0.4 - x_C1_0.44 >= 0
x_C1_0.44 - x_C1_0.45 >= 0
x_C1_0.45 - x_C1_0.48 >= 0
x_C1_0.48 - x_C1_0.59 >= 0
x_C1_0.59 - x_C1_0.6 >= 0
x_C1_0.6 - x_C1_0.61 >= 0
x_C1_0.61 - x_C1_0.62 >= 0
x_C1_0.62 - x_C1_0.64 >= 0
x_C1_0.64 - x_C1_0.65 >= 0
x_C1_0.65 - x_C1_0.66 >= 0
x_C1_0.66 - x_C1_0.68 >= 0
x_C1_0.68 - x_C1_0.69 >= 0
x_C1_0.69 - x_C1_0.71 >= 0
x_C1_0.71 - x_C1_0.73 >= 0
x_C1_0.73 - x_C1_0.74 >= 0
x_C1_0.74 - x_C1_0.76 >= 0
x_C1_0.76 - x_C1_0.78 >= 0
x_C1_0.78 - x_C1_0.8 >= 0
x_C1_0.8 - x_C1_0.83 >= 0
x_C1_0.83 - x_C1_0.87 >= 0
x_C1_0.87 - x_C1_1.0 >= 0
x_C2_0.03 - x_C2_0.06 >= 0
x_C2_0.06 - x_C2_0.22 >= 0
x_C2_0.22 - x_C2_0.24 >= 0
x_C2_0.24 - x_C2_0.25 >= 0
x_C2_0.25 - x_C2_0.27 >= 0
x_C2_0.27 - x_C2_0.3 >= 0
x_C2_0.3 - x_C2_0.4 >= 0
x_C2_0.4 - x_C2_0.44 >= 0
x_C2_0.44 - x_C2_0.45 >= 0
x_C2_0.45 - x_C2_0.49 >= 

In [74]:
problem += epsilon

In [75]:
problem.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/lib/python3.11/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/f6ac81694bef40c7965a3d312fc9c68a-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/f6ac81694bef40c7965a3d312fc9c68a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 82 COLUMNS
At line 255 RHS
At line 333 BOUNDS
At line 412 ENDATA
Problem MODEL has 77 rows, 78 columns and 170 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 39 (-38) rows, 41 (-37) columns and 88 (-82) elements
Perturbing problem by 0.001% of 1 - largest nonzero change 0 ( 0%) - largest zero change 9.3128486e-05
0  Obj 3.1 Primal inf 6.1999981 (2)
31  Obj 1.9985824 Primal inf 3.999996 (4)
39  Obj 0.49972225
Optimal - objective value 0.5
After Postsolve, objective 0.5, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 0.5 - 39 iterations

1

In [76]:
#Retrieve the status of the solution:
status = LpStatus[problem.status]
print("Status:", status)

#Retrieve the solution as a dictionary with variable names and their values:
solution = {variable.name: variable.varValue for variable in problem.variables()}
print("Solution:", solution)

Status: Optimal
Solution: {'epsilon': 0.5, 'x_C1_0.32': 0.0, 'x_C1_0.34': 0.0, 'x_C1_0.35': 0.0, 'x_C1_0.4': 0.0, 'x_C1_0.44': 0.0, 'x_C1_0.45': 0.0, 'x_C1_0.48': 0.0, 'x_C1_0.59': 0.0, 'x_C1_0.6': 0.0, 'x_C1_0.61': 0.0, 'x_C1_0.62': 0.0, 'x_C1_0.64': 0.0, 'x_C1_0.65': 0.0, 'x_C1_0.66': 0.0, 'x_C1_0.68': 0.0, 'x_C1_0.69': 0.0, 'x_C1_0.71': 0.0, 'x_C1_0.73': 0.0, 'x_C1_0.74': 0.0, 'x_C1_0.76': 0.0, 'x_C1_0.78': 0.0, 'x_C1_0.8': 0.0, 'x_C1_0.83': 0.0, 'x_C1_0.87': 0.0, 'x_C1_1.0': 0.0, 'x_C2_0.03': 0.0, 'x_C2_0.06': 0.0, 'x_C2_0.22': 0.0, 'x_C2_0.24': 0.0, 'x_C2_0.25': 0.0, 'x_C2_0.27': 0.0, 'x_C2_0.3': 0.0, 'x_C2_0.4': 0.0, 'x_C2_0.44': 0.0, 'x_C2_0.45': 0.0, 'x_C2_0.49': 0.0, 'x_C2_0.54': 0.0, 'x_C2_0.55': 0.0, 'x_C2_0.83': 0.0, 'x_C2_0.86': 0.0, 'x_C2_0.87': 0.0, 'x_C2_0.9': 0.0, 'x_C2_0.91': 0.0, 'x_C2_0.93': 0.0, 'x_C2_0.95': 0.0, 'x_C2_0.97': 0.0, 'x_C2_1.0': 0.0, 'x_C3_0.0': 0.5, 'x_C3_0.38': 0.5, 'x_C3_0.45': 0.0, 'x_C3_0.54': 0.0, 'x_C3_0.56': 0.0, 'x_C3_0.57': 0.0, 'x_C3_0.65':